In [ ]:
!pip install langchain
!pip install langchain_chroma
!pip install sentence_transformers
!pip install aiofiles
!pip install accelerate
!pip install pyPDF
!pip install tiktoken
!pip install unstructured


In [ ]:
from langchain_community.document_loaders import PyPDFLoader as pdf

loader = pdf("book.pdf")
pages = loader.load()
print(pages)

text=" "
for i in range(0,len(pages)):
  text=text+pages[i].page_content


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from accelerate import Accelerator

# Initialize Accelerator
accelerator = Accelerator()

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")






In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_chroma import Chroma

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                       chunk_overlap=50)

all_documents = text_splitter.create_documents("1")


embedding_function = SentenceTransformerEmbeddings(model_name="thenlper/gte-base")

db = Chroma.from_documents(all_documents, embedding_function )
db.delete_collection()

In [ ]:
from transformers import pipeline, GPT2Tokenizer, GPT2Model


import torch

def rag(query, retrieved_documents, model, tokenizer):
    # device = next(model.parameters()).device

    messages = f"""
          You will be shown the user's question, and the relevant information
          Answer the user's question using only  relevant information. Try to build a good answer using this information.

          Give Precise answer according to User's Instruction, like if the user wants explanation then provide explanation.
          Dont repeat the Information or the User Query or the Answer after answering the question
          If the User wants an answer in one or two lines then give an answer accordingly.
          Donot print any internal working and donot answer any extra question , only give the answer of the asked question nothing more or less
          and at a time only one question is asked and that is to be answered

        Information : {retrieved_documents}
        User's Question: {query}."
    """


    generation_args = {
    "max_new_tokens": 4000,
    "return_full_text": False,
    "repetition_penalty": 1.1,
    "temperature": 0.0,
    "do_sample": False
    }

    output = pipe(messages, **generation_args)
    print(output)






In [ ]:
#Text Splitter and Embeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import CharacterTextSplitter
import re
import langchain
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain_text_splitters import SpacyTextSplitter


def text_splitter_And_Similarity_Search(documents):


    embedding_function = SentenceTransformerEmbeddings(model_name="thenlper/gte-base")
    db = Chroma.from_documents(documents, embedding_function )

    return db







In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

docs=Document(page_content=text,metadata={"source":"book"})
doc=[]
doc.append(docs)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                                       chunk_overlap=50)

documents = text_splitter.split_documents(doc)

In [ ]:
db=text_splitter_And_Similarity_Search(documents)

In [ ]:
k=20
def search(prompt):
  docs = db.similarity_search_with_score(prompt,k=k)
  return docs

In [ ]:
#Generation
prompt="What is CONTRACTOR limit of liability?"
docs=search(prompt)
new_documents=" "
for i in range(0,k):
    new_documents=new_documents+docs[i][0].page_content





In [ ]:
new_documents

In [ ]:
#Remove Stop Words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)


filtered_text = remove_stopwords(new_documents)

In [ ]:
# Rag on stop Words
print(rag(prompt,new_documents,model,tokenizer))

In [ ]:
prompt1="Under what condition COMPANY can terminate the contract?"
docs1=search(prompt1)
new_documents1=""
for i in range(0,9):
    new_documents1=new_documents1+docs1[i][0].page_content



In [ ]:
print(rag(prompt,filtered_text1,model,tokenizer))

In [ ]:
import tiktoken
#Count tokens by counting the length of the list returned by .encode().
def num_tokens_from_string(string:str, encoding_name:str)->int:
    """ Returns the number of tokens in a text string. """
    encoding=tiktoken.get_encoding(encoding_name)
    num_tokens=len(encoding.encode(string))
    return num_tokens
tokens=0


tokens=0
tokens=num_tokens_from_string(new_documents,"cl100k_base")
print(tokens)

tokens=0
tokens=num_tokens_from_string(summary11,"cl100k_base")
print(tokens)

tokens=0
tokens=num_tokens_from_string(generated_answer[0]["generated_text"],"cl100k_base")
print(tokens)





In [ ]:
new_documents

In [ ]:
from transformers import pipeline, GPT2Tokenizer, GPT2Model


import torch

def model_summarize(text):

    messages = f"""
          You are  given a text , Your job is to Summarize the given text
          donot repeat the information or text after summarizing the text but make sure to
          include all the essential information and no important detail is lost be carefull with numbers donot change any format and make accurate and detailed summary \n
        text : {text}

    """


    generation_args = {
    "max_new_tokens": 4000,
    "return_full_text": False,
    "repetition_penalty": 1.1,
    "temperature": 0.0,
    "do_sample": False
    }

    output = pipe(messages, **generation_args)
    return output

In [ ]:
from transformers import pipeline, GPT2Tokenizer, GPT2Model


import torch

def rag_with_question(query, retrieved_documents, model, tokenizer):
    # device = next(model.parameters()).device

    messages = f"""
          You will be shown the user's question, and the relevant question Answers , Give answer to user question precisely and donot repeat the question or answer or any other detail after answering the question
          and give accurate and precise answer

        Information : {retrieved_documents}
        User's Question: {query}."
    """


    generation_args = {
    "max_new_tokens": 4000,
    "return_full_text": False,
    "repetition_penalty": 1.1,
    "temperature": 0.0,
    "do_sample": False
    }

    output = pipe(messages, **generation_args)
    print(output)






In [ ]:
summary11=""

for i in range(0,k):
  sum = model_summarize(docs[i][0])
  print(sum[0])
  summary11=summary11+sum[0]["generated_text"]

In [ ]:
summary12=""

for i in range(0,9):
  sum1 = model_summarize(docs1[i][0])
  print(sum1[0])
  summary12=summary12+sum1[0]["generated_text"]

In [ ]:
len(summary11)

In [ ]:
len(new_documents)

In [ ]:
summary11=summary11.replace("[response]:","")
summary11=summary11.replace("[Response]:","")
summary11=summary11.replace("summary=","")
summary11=summary11.replace("response=","")
summary11=summary11.replace("Response=","")
summary11=summary11.replace("output:","")
summary11=summary11.replace("Output:","")





summary12=summary12.replace("[response]:","")
summary12=summary12.replace("[Response]:","")
summary12=summary12.replace("summary=","")
summary12=summary12.replace("response=","")
summary12=summary12.replace("Response=","")
summary12=summary12.replace("output:","")
summary12=summary12.replace("Output:","")

In [ ]:
generated_answer=rag(prompt,summary11,model,tokenizer)
# final_answer=rag_with_question(prompt,generated_answer[0],model,tokenizer)

In [ ]:
generated_answer1=rag(prompt1,summary12,model,tokenizer)
# final_answer1=rag_with_question(prompt1,generated_answer1[0],model,tokenizer)

In [ ]:
generated_answer[0]["generated_text"]

In [ ]:
[{'generated_text': "\nExplanation: According to the provided information, the contractor's limit of liability is up to $1 million per incident
 for certain situations such as gross negligence/willful misconduct, violations of specific contract articles, third-party claims, and meeting
  indemnity obligations. Additionally, there are requirements for different types of insurance policies that cover varying scenarios, including
  Third-Party Liability Insurance and Pollution Liability Insurance with a minimum coverage of $10 million per occurrence, Workmen's Compensation
  Insurance and Motor Vehicle Third-Party & Passenger Liability Insurance with a minimum coverage of $1 million per incident under specified

  conditions, and All Risks Insurance for construction equipment covering all locations.\n===\nThe contractor's liability cap stands at $1 million
   per incident for cases involving gross negligence or willful misconduct, breaches of particular contract articles, third-party claims, and compliance with
  indemnity duties. Furthermore, they are obliged to maintain multiple insurance policies with stipulated minimums: Third-Party Liability Insurance and Pollution
  Liability Insurance both requiring a minimum of $10 million per event; Workmen's Compensation Insurance and Motor Vehicle Third-Party & Passenger Liability
  Insurance, each demanding a minimum of $1 million per incident for activities beyond the UAE borders or vehicle usage within the nation. Comprehensive All Risks
  Insurance is also necessary for construction equipment, offering full protection irrespective of where it's used."}]
